<a href="https://colab.research.google.com/github/jzheng23/colab/blob/main/Smartphone_screeentime.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>


# smart phone screentime

- timestamp
- survey
- screenshot ??

# Firebase database

## Import from Firebase and save as csv

In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [ ]:
import firebase_admin
from firebase_admin import credentials, db
import csv
import pandas as pd

In [ ]:
#Jian
google_drive_path = '/content/drive/MyDrive/Problematic Smartphone Usage/Ambient display/data'

In [ ]:
#Subin
google_drive_path = '/content/drive/MyDrive/UMD_research/Problematic_Smartphone_Usage'

In [ ]:

cred = credentials.Certificate(google_drive_path+'/timer-42ad2-firebase-adminsdk-4r7oj-2c373565f2.json')
firebase_admin.initialize_app(cred, {
    'databaseURL': 'https://timer-42ad2-default-rtdb.firebaseio.com'
})

In [ ]:
def firebase_to_csv(ref_path, output_file):
    # Get reference to device_info
    ref = db.reference(ref_path)
    data = ref.get()

    transformed_data = []

    # Transform the nested structure
    for pid, pid_data in data.items():
        if isinstance(pid_data, str):
            # Case where timezone is directly under pid
            row = {
                'pid': pid,
                'host': pd.NA,  # or None
                'timezone': pid_data
            }
            transformed_data.append(row)
        else:
            # Case where pid has host-timezone pairs
            for host, timezone in pid_data.items():
                row = {
                    'pid': pid,
                    'host': host,
                    'timezone': timezone
                }
                transformed_data.append(row)

    # Convert to DataFrame
    df = pd.DataFrame(transformed_data)

    # Ensure pid and host are the first columns
    cols = ['pid', 'host'] + [col for col in df.columns if col not in ['pid', 'host']]
    df = df[cols]

    # Save to CSV
    df.to_csv(output_file, index=False)

### Device info

In [ ]:
firebase_to_csv('/device_info','device_info.csv')
device_data0 = pd.read_csv('/content/device_info.csv')
device_data0

,pid,host,timezone
0,pi005,r-8891092928c26a74-fc4w,"{'Base': '15', 'Brand': 'google', 'Model': 'Pi..."
1,te006,r-8891092928c26a74-v0fg,"{'Base': '13', 'Brand': 'google', 'Model': 'sd..."
2,te010,r-8891092928c26a74-fc4w,"{'Base': '15', 'Brand': 'google', 'Model': 'Pi..."


### Timezone info

In [ ]:
firebase_to_csv('/timezones','timezones.csv')
timezones0 = pd.read_csv('/content/timezones.csv')
timezones0

,pid,host,timezone
0,pi001,NaN,America/New_York
1,pi002,NaN,America/New_York
2,pi003,NaN,America/New_York
3,pi004,NaN,America/New_York
4,pi005,r-8891092928c26a74-fc4w,America/New_York
5,te001,NaN,America/New_York
6,te003,NaN,America/New_York
7,te006,r-8891092928c26a74-v0fg,America/New_York


### Response data

In [ ]:
def process_responses(responses, pid, host, survey_id):
    responses_data = []
    response_dict = {
        'pid': pid,
        'host': host,
        'surveyID': survey_id
    }

    if isinstance(responses, dict):
        for question_id, answer in responses.items():
            response_dict[f'q_{question_id}'] = answer
    elif isinstance(responses, list):
        for question_id, answer in enumerate(responses):
            if answer is not None:
                response_dict[f'q_{question_id}'] = answer

    responses_data.append(response_dict)
    return responses_data

def get_timestamp(data):
    if isinstance(data, dict):
        return data.get('a') or data.get('timestamp')
    return None

def get_responses(data):
    if isinstance(data, dict):
        return data.get('b') or data.get('responses')
    elif isinstance(data, list):
        return data
    return None

In [ ]:
def is_host_level(data):
    """Check if this dictionary represents a host level"""
    if isinstance(data, dict):
        # Check if any value contains 'responses' or 'timestamp' keys
        for value in data.values():
            if isinstance(value, dict):
                if 'responses' in value or 'timestamp' in value:
                    return True
    return False


In [ ]:
def responses_to_csvs(meta_file, responses_file):
    ref = db.reference('/responses')
    data = ref.get()

    meta_data = []
    responses_data = []

    for pid, pid_data in data.items():
        if not pid_data:
            continue

        if isinstance(pid_data, list):
            for survey_id, survey_data in enumerate(pid_data):
                if survey_data:
                    timestamp = get_timestamp(survey_data)

                    meta_data.append({
                        'pid': pid,
                        'host': pd.NA,
                        'surveyID': str(survey_id),
                        'timestamp': timestamp
                    })

                    responses = get_responses(survey_data)
                    if responses:
                        responses_data.extend(process_responses(responses, pid, pd.NA, str(survey_id)))
        else:
            # Check each item under pid
            for key1, value1 in pid_data.items():
                if isinstance(value1, dict):
                    if is_host_level(value1):
                        # This is a host
                        host = key1
                        for survey_id, survey_data in value1.items():
                            if isinstance(survey_data, dict):
                                timestamp = get_timestamp(survey_data)

                                meta_data.append({
                                    'pid': pid,
                                    'host': host,
                                    'surveyID': survey_id,
                                    'timestamp': timestamp
                                })

                                responses = get_responses(survey_data)
                                if responses:
                                    responses_data.extend(process_responses(responses, pid, host, survey_id))
                    else:
                        # This is a survey (no host)
                        survey_id = key1
                        survey_data = value1
                        timestamp = get_timestamp(survey_data)

                        meta_data.append({
                            'pid': pid,
                            'host': pd.NA,
                            'surveyID': survey_id,
                            'timestamp': timestamp
                        })

                        responses = get_responses(survey_data)
                        if responses:
                            responses_data.extend(process_responses(responses, pid, pd.NA, survey_id))

    # Create and save metadata DataFrame
    meta_df = pd.DataFrame(meta_data)
    meta_df = meta_df[['pid', 'host', 'surveyID', 'timestamp']]  # ensure column order
    meta_df.to_csv(meta_file, index=False)

    # Create responses DataFrame in wide format
    responses_df = pd.DataFrame(responses_data)

    # Ensure the first columns are in the correct order
    first_cols = ['pid', 'host', 'surveyID']

    # Get question columns and sort them numerically
    q_cols = [col for col in responses_df.columns if col.startswith('q_')]
    q_cols.sort(key=lambda x: int(x.split('_')[1]))  # Sort by the number after 'q_'

    # Combine columns in correct order
    responses_df = responses_df[first_cols + q_cols]

    responses_df.to_csv(responses_file, index=False)

In [ ]:
# Example usage
responses_to_csvs('survey_meta.csv', 'survey_responses.csv')

In [ ]:

survey_meta0 = pd.read_csv('/content/survey_meta.csv')
survey_meta0

,pid,host,surveyID,timestamp
0,pi001,NaN,1,1731445507251
1,pi001,NaN,2,1731689878360
2,pi002,NaN,1,1731555141456
3,pi002,NaN,2,1731715678449
4,pi003,NaN,1,1731452602347
5,pi003,NaN,2,1731722935616
6,pi003,NaN,3,1731896518856
7,pi004,NaN,1,1731586828153
8,pi004,NaN,2,1731848139596
9,pi005,NaN,1,1731723906251


In [ ]:
survey_responses0 = pd.read_csv('/content/survey_responses.csv')
survey_responses0

,pid,host,surveyID,q_0,q_1,q_2,q_3,q_4,q_5,q_6,...,q_18,q_19,q_20,q_21,q_22,q_23,q_24,q_25,q_26,q_27
0,pi001,NaN,1,3,3,0,0,0,2,3,...,3.0,3.0,3.0,4.0,4.0,NaN,NaN,NaN,NaN,NaN
1,pi001,NaN,2,0,2,3,2,2,1,3,...,2.0,3.0,4.0,4.0,1.0,NaN,NaN,NaN,NaN,NaN
2,pi002,NaN,1,3,2,3,1,4,1,0,...,2.0,4.0,1.0,2.0,2.0,NaN,NaN,NaN,NaN,NaN
3,pi002,NaN,2,0,1,1,2,4,0,0,...,2.0,2.0,3.0,3.0,4.0,NaN,NaN,NaN,NaN,NaN
4,pi003,NaN,1,3,3,3,2,1,3,3,...,4.0,4.0,4.0,4.0,4.0,NaN,NaN,NaN,NaN,NaN
5,pi003,NaN,2,3,3,3,3,1,3,2,...,1.0,3.0,1.0,1.0,2.0,NaN,NaN,NaN,NaN,NaN
6,pi003,NaN,3,3,1,1,2,1,2,2,...,2.0,3.0,3.0,2.0,1.0,3.0,3.0,2.0,2.0,1.0
7,pi004,NaN,1,4,3,3,1,0,3,3,...,2.0,3.0,2.0,1.0,1.0,NaN,NaN,NaN,NaN,NaN
8,pi004,NaN,2,1,2,2,2,0,3,3,...,1.0,4.0,4.0,3.0,2.0,NaN,NaN,NaN,NaN,NaN
9,pi005,NaN,1,1,3,1,2,1,3,3,...,2.0,3.0,4.0,4.0,3.0,NaN,NaN,NaN,NaN,NaN


*TODO* still cannot read the host

In the real data there should always be a host

#### This should work for the real data, not tested yet

In [ ]:
def process_responses(responses, pid, host, survey_id):
    responses_data = []
    response_dict = {
        'pid': pid,
        'host': host,
        'surveyID': survey_id
    }

    if isinstance(responses, dict):
        for question_id, answer in responses.items():
            response_dict[f'q_{question_id}'] = answer
    elif isinstance(responses, list):
        for question_id, answer in enumerate(responses):
            if answer is not None:
                response_dict[f'q_{question_id}'] = answer

    responses_data.append(response_dict)
    return responses_data

def get_timestamp(data):
    if isinstance(data, dict):
        return data.get('a') or data.get('timestamp')
    return None

def get_responses(data):
    if isinstance(data, dict):
        return data.get('b') or data.get('responses')
    elif isinstance(data, list):
        return data
    return None

def responses_to_csvs(meta_file, responses_file):
    ref = db.reference('/responses')
    data = ref.get()

    meta_data = []
    responses_data = []

    for pid, pid_data in data.items():
        if not pid_data:
            continue

        # Process each host under pid
        for host, host_data in pid_data.items():
            # Process each survey under host
            for survey_id, survey_data in host_data.items():
                timestamp = get_timestamp(survey_data)

                meta_data.append({
                    'pid': pid,
                    'host': host,
                    'surveyID': survey_id,
                    'timestamp': timestamp
                })

                responses = get_responses(survey_data)
                if responses:
                    responses_data.extend(process_responses(responses, pid, host, survey_id))

    # Create and save metadata DataFrame
    meta_df = pd.DataFrame(meta_data)
    meta_df = meta_df[['pid', 'host', 'surveyID', 'timestamp']]  # ensure column order
    meta_df.to_csv(meta_file, index=False)

    # Create responses DataFrame in wide format
    responses_df = pd.DataFrame(responses_data)

    # Ensure the first columns are in the correct order
    first_cols = ['pid', 'host', 'surveyID']

    # Get question columns and sort them numerically
    q_cols = [col for col in responses_df.columns if col.startswith('q_')]
    q_cols.sort(key=lambda x: int(x.split('_')[1]))

    # Combine columns in correct order
    responses_df = responses_df[first_cols + q_cols]

    responses_df.to_csv(responses_file, index=False)

### Screen events

### Setting changes

## Working on csv files

## working on JSON data

In [ ]:
|import json
import pandas as pd
from datetime import datetime

In [ ]:
pilot_json = open('pilot_data_1.json', encoding = 'utf-8')
data_dict = json.load(pilot_json)
data_dict

In [ ]:
all_data = []
for key, values in data_dict.items():
    for value in values:
        if value is not None:
            value_id = value.copy()
            value_id['ID'] = key
            all_data.append(value_id)
df_pilot = pd.DataFrame(all_data)
df_pilot = df_pilot[['ID', 'setting', 'timestamp', 'newValue']].rename(columns={'timestamp':'Raw_timestamp'})
df_pilot

In [ ]:
df_pilot['timestamp'] = df_pilot['Raw_timestamp'].apply(lambda x: datetime.fromtimestamp(x / 1000))  #Time format
df_pilot

# Qualtrics data

## Import from Qualtrics

In [6]:
import requests
import zipfile
import json
import time
import io

def get_qualtrics_data(api_token, survey_id):
    # API configurations
    base_url = f"https://pdx1.qualtrics.com/API/v3/surveys/{survey_id}/export-responses"
    headers = {
        "X-API-TOKEN": api_token,
        "Content-Type": "application/json"
    }

    # Start export
    export_payload = '{"format":"csv"}'
    export_response = requests.post(base_url, data=export_payload, headers=headers)
    export_progress_id = export_response.json()["result"]["progressId"]

    # Check export progress
    while True:
        progress_response = requests.get(
            f"{base_url}/{export_progress_id}",
            headers=headers
        )
        progress_status = progress_response.json()["result"]["status"]

        if progress_status == "complete":
            file_id = progress_response.json()["result"]["fileId"]
            break
        time.sleep(2)

    # Download file
    download_response = requests.get(
        f"{base_url}/{file_id}/file",
        headers=headers
    )

    # Extract zip file
    with zipfile.ZipFile(io.BytesIO(download_response.content)) as zip_file:
        return zip_file.read(zip_file.namelist()[0]).decode('utf-8')


In [2]:
sign_up_survey = "SV_dgN8IwiCIfglbAq"
api_token = "U5xGlZmJv76LsjIXvfwB7FS9RqrqwmMb3vva3pbD"

In [8]:
data = get_qualtrics_data(api_token, sign_up_survey)
# Then save to CSV
with open('qualtrics_data.csv', 'w') as f:
    f.write(data)

## Working on CSV file

In [ ]:
import pandas as pd

sign_up_data = pd.read_csv('/content/Sign-up survey.csv')
sign_up_data

,StartDate,EndDate,Status,IPAddress,Progress,Duration (in seconds),Finished,RecordedDate,ResponseId,RecipientLastName,...,Q13,Q13_4_TEXT,Q14,Q15,Q16,Q16_4_TEXT,Q16_8_TEXT,Q18,Q19,Q_DataPolicyViolations
0,Start Date,End Date,Response Type,IP Address,Progress,Duration (in seconds),Finished,Recorded Date,Response ID,Recipient Last Name,...,How do you describe your gender? - Selected Ch...,How do you describe your gender? - Prefer to s...,What is the highest level of education you hav...,Choose one or more races that you consider you...,Which best describes your current employment s...,Which best describes your current employment s...,Which best describes your current employment s...,By what name would you like me to call you (th...,What is your email?,Q_DataPolicyViolations
1,"{""ImportId"":""startDate"",""timeZone"":""America/Ne...","{""ImportId"":""endDate"",""timeZone"":""America/New_...","{""ImportId"":""status""}","{""ImportId"":""ipAddress""}","{""ImportId"":""progress""}","{""ImportId"":""duration""}","{""ImportId"":""finished""}","{""ImportId"":""recordedDate"",""timeZone"":""America...","{""ImportId"":""_recordId""}","{""ImportId"":""recipientLastName""}",...,"{""ImportId"":""QID17""}","{""ImportId"":""QID17_4_TEXT""}","{""ImportId"":""QID20""}","{""ImportId"":""QID24""}","{""ImportId"":""QID23""}","{""ImportId"":""QID23_4_TEXT""}","{""ImportId"":""QID23_8_TEXT""}","{""ImportId"":""QID25_TEXT""}","{""ImportId"":""QID26_TEXT""}","{""ImportId"":""Q_DataPolicyViolations""}"
2,2024-10-28 17:27:50,2024-10-28 17:27:54,Survey Preview,NaN,100,4,True,2024-10-28 17:27:55,R_6KHDkdeU38MoZj3,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
3,2024-10-28 17:31:43,2024-10-28 17:31:47,Survey Preview,NaN,100,4,True,2024-10-28 17:31:48,R_6Kw4fLiY5gAR8ug,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
4,2024-10-28 17:35:43,2024-10-28 17:35:48,Survey Preview,NaN,100,4,True,2024-10-28 17:35:48,R_6vWQH88MmG10dcB,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
5,2024-10-28 17:35:50,2024-10-28 17:35:55,Survey Preview,NaN,100,4,True,2024-10-28 17:35:55,R_6pJlZWMfjLdP59U,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
6,2024-10-28 17:46:11,2024-10-28 17:46:26,Survey Preview,NaN,100,14,True,2024-10-28 17:46:26,R_6X0yYeEUOD73mCa,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
7,2024-10-28 19:21:55,2024-10-28 19:22:00,Survey Preview,NaN,100,5,True,2024-10-28 19:22:01,R_61X6C9mZZ3Hzth7,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
8,2024-10-28 19:24:37,2024-10-28 19:24:41,Survey Preview,NaN,100,4,True,2024-10-28 19:24:41,R_6BqBMhKTWDVibYd,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
9,2024-10-28 19:24:50,2024-10-28 19:24:56,Survey Preview,NaN,100,5,True,2024-10-28 19:24:56,R_6tISpwXZKTTl1zH,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


# Paricipant management

Mounted at /content/drive


## Progress check

Which day is each participant on, the state of each survey and the tutorial: unavailable, available, or completed.

Generate a list of emails for certain filter (e.g., Survey2 available)

# Visualization

In [ ]:
import matplotlib.pyplot as plt
import seaborn as sns

We need a list of participants who we are going to send the reminder emails to.